In [ ]:
import pandas as pd
import numpy as np
import os 
import pickle
from fuzzywuzzy import process, fuzz
from rapidfuzz import process, utils
from collections import Counter
pd.options.display.max_columns = None

# checking the dataset's data
Checking the data from the [given](https://www.kaggle.com/datasets/andrewmvd/spotify-playlists?select=spotify_dataset.csv) database.

In [ ]:
data = pd.read_csv('../data/spotify_dataset.csv', sep = ',', on_bad_lines='skip')
print(data.shape)
data.head()

check if data has duplicated values

In [ ]:
data.columns

In [ ]:
data[['user_id',	' "artistname"',	' "trackname"']].duplicated().sum()

In [ ]:
data = data.drop_duplicates(subset=[' "artistname"',	' "trackname"'])
print(data.shape)
data.head()

remove unnecesary columns

In [ ]:
data_new = data.drop(['user_id', ' "playlistname"'], axis = 1)
print(data_new.shape)
data_new.head()

# check if the data is clean
It is possible that some artist are named similarly and create duplicated values

In [ ]:
data_new.dropna(subset=[' "artistname"'], inplace=True)
data_new.dropna(subset=[' "trackname"'], inplace=True)
print(data_new.shape)
data_new.head()

In [ ]:
data_new['artist_clean'] = data_new[' "artistname"'].str.lower().str.replace('&', 'and').str.replace('  ', ' ')
print(len(data_new['artist_clean'].unique().tolist()), len(data_new[' "artistname"'].unique().tolist()))

In [ ]:
def sacar_parecido(col, dict_artistas):   
    maximo = 0
    for key in dict_artistas.keys():
    #print(key)
        #print(genero)
        comparo = fuzz.ratio(key, col)
        if comparo > maximo:
            maximo = comparo
            resultado = key
    if maximo > 90:
        return resultado
    else:
        return np.nan

In [ ]:
data_new[data_new['artist_clean'].str.contains('costello') == True]['artist_clean'].unique().tolist()

In [ ]:
cuenta_artistas = Counter(artistas for artistas in data_new['artist_clean'])
print(len(cuenta_artistas))
cuenta_artistas

In [ ]:
dict_artistas = dict(cuenta_artistas)
dict_artistas = {k:v for k,v in dict_artistas.items() if v > 4}
print(len(dict_artistas))
dict_artistas

In [ ]:
data_new['artist'] = data_new.apply(lambda x: sacar_parecido(x['artist_clean'], dict_artistas), axis=1)
data_new['artist'].value_counts()

In [ ]:
data_new[' "artistname"'].unique().tolist()

# Conversion
I will save the input file as .pkl as the original one is too big for github.

In [ ]:
with open('../data/spotify_dataset.pickle', 'wb') as data_nowplaying:
    pickle.dump(data_new, data_nowplaying)